# Variables

In [1]:
#https://www.tensorflow.org/api_docs/python/tf/Variable

In [2]:
import tensorflow as tf

In [3]:
v = tf.Variable(1.)
print(v)
v.assign(2.)
print(v)
v.assign_add(0.5)
print(v)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.5>


In [4]:
v = tf.constant(1.)
print(v)
v.assign(2.)
print(v)


tf.Tensor(1.0, shape=(), dtype=float32)


AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

- tf.constant : are fixed values, and hence not trainable.
- tf.Variable: these are tensors (arrays) that were initialized in a session and are trainable (with trainable i mean this can be optimized and can changed over time)

In [ ]:
#https://www.tensorflow.org/api_docs/python/tf/GradientTape

In [6]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    y = x * x
dy_dx = g.gradient(y, x)
print(dy_dx)

None


In [7]:
x = tf.Variable(3.0)
with tf.GradientTape() as g:
    y = x * x
dy_dx = g.gradient(y, x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [8]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
  g.watch(x)
  y = x * x
dy_dx = g.gradient(y, x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


Trainable variables (created by tf.Variable or tf.compat.v1.get_variable, where trainable=True is default in both cases) are automatically watched. Tensors can be manually watched by invoking the watch method on this context manager.

# tf.function and Autograph

tf.function allows you to switch from eager execution to graph execution

tf.function uses a library called AutoGraph (tf.autograph) to convert Python code into graph-generating code.

In [ ]:
#https://www.tensorflow.org/guide/intro_to_graphs
#https://towardsdatascience.com/eager-execution-vs-graph-execution-which-is-better-38162ea4dbf6

Why do we need graphs?
As you already read above, eager execution is a better choice for easy debugging and more intuitive programming using Python. It is the same as writing regular python code, where you can run your code line by line in console, or as a script

However, running TensorFlow code step-by-step (as in eager execution) in Python prevents a host of accelerations otherwise available in the lazy mode. If you can extract tensor computations from Python, you can make them into a graph

Graphs are a type of data structures that contains tensors and the computations performed. Graphs store the flow of information and operations between tensors through tf.Operation objects and tf.Tensor tensors.

But, why use Graphs?


The primary reason is, graphs allow
your neural network model to be used in environments that dont have a Python interpreter. For example, graphs can be deployed in mobile applications or servers. This is not suitable for eagerly executed code.
The second reason is that graphs can speed up computation time. They eliminate the need for repetitive initialisation of variables and computation on these variables.

The tf.function API is used in TF2.0 to create graphs for eagerly executed code.

There are two ways you can use this.
1. As a decorator: Using @tf.function decorator before your code will create a graph for that piece of code.

In [ ]:
import tensorflow as tf
import time
from datetime import datetime

@tf.function
def function(x):
  a = tf.constant([[2.0], [3.0]])
  b = tf.constant(4.0)
  return a+b

In [ ]:
# Plot a graph for function() using Tensorboard
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = function(2)
with writer.as_default():
  tf.summary.trace_export(
      name="function_trace",
      step=0,
      profiler_outdir=logdir)

In [5]:
%load_ext tensorboard
%tensorboard --logdir logs/func

2. As a callable function : In this method you can simply tf.function-ise an existing function to create a graph for that function.

In [ ]:
# Define a Python function
def callable_func(a, b):
  return tf.matmul(a, b)

# Create a `Function` object that contains a graph
function_that_uses_graph = tf.function(callable_func)

# Make some tensors
a1 = tf.constant([[1.0, 2.0]])
b1 = tf.constant([[2.0], [3.0]])

# It works!
function_that_uses_graph(a1, b1).numpy()